In [52]:
import os

# Define input folder
input_folder_path = r"100notes_txt"

# dflow: data flow that passes through the pipeline
# dflow has filenames as keys; 
# values start with the original text, and are updated 
# to a list that includes results from each pipe
dflow = {}

### Pulling model to Ollama

In [53]:
!ollama pull gemma3:27b
!ollama pull llama3.1:8b
!ollama pull mistral-small3.1:latest

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling e796792eba26... 100% ▕████████████████▏  17 GB                         
pulling e0a42594d802... 100% ▕████████████████▏  358 B                         
pulling dd084c7d92a3... 100% ▕████████████████▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕████████████████▏   77 B                         
pulling f838f048d368... 100% ▕████████████████▏  490 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 667b0c1932bc... 100% ▕████████████████▏ 4.9 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 455f34728c9b... 100% ▕████████████████▏  487 B                        

### Each time you will run the whole pipeline with one LLM. Follow the instruction below:

- Change the first line of 'ros_extract' and 'ros_classify' file with the model name <br> - For example, if you run gemma3:27b, change the first line to: <br>FROM gemma3:27b <br> - Similar with other models:<br>FROM llama3.1:8b <br> FROM mistral-small3.1:latest

After you change the files content, run create_model.sh to create ros_extract and ros_classify models based on the corresponding LLM you want to use

In [54]:
!source create_model.sh

gathering model components 
using existing layer sha256:1fa8532d986d729117d6b5ac2c884824d0717c9468094554fd1d36412c740cfc 
using existing layer sha256:6db27cd4e277c91264572b9c899c1980daa8dea11e902f0070a6f4763f3d13c8 
using existing layer sha256:78f1631bbae9c1601f7ef958c1a2ded966766a1601ca3a4f8b29d0a0c2c60222 
creating new layer sha256:77666e5fb92b1205d96fa2c71c8cadfbf56d2ad82636313603ec0113accfd1e1 
writing manifest 
success 
gathering model components 
using existing layer sha256:1fa8532d986d729117d6b5ac2c884824d0717c9468094554fd1d36412c740cfc 
using existing layer sha256:6db27cd4e277c91264572b9c899c1980daa8dea11e902f0070a6f4763f3d13c8 
using existing layer sha256:e112f3568ab826a6f1f317c70809ca002dad47947352d337c7b2b1bbf995df28 
using existing layer sha256:77666e5fb92b1205d96fa2c71c8cadfbf56d2ad82636313603ec0113accfd1e1 
writing manifest 
success 


Set the output folder to corresponding model

In [55]:
# output_folder_path = r"Output_gemma3_27b"
output_folder_path = r"Output_mistral-small3.1"
# output_folder_path = r"Output_llama3.1_8b"

if os.path.exists(output_folder_path) == False:
    os.makedirs(output_folder_path)

# PIPELINE

## Pipe 1: Load notes

In [56]:
for filename in os.listdir(input_folder_path):
    if filename.endswith('.txt'):  # Process only .txt files
        file_path = os.path.join(input_folder_path, filename)
        with open(file_path) as f:
            dflow[filename]=f.read()

print(f'{len(dflow)} notes loaded')
dflow

100 notes loaded


{'sample_71.txt': 'Sample Type / Medical Specialty:  General Medicine\nSample Name: Gen Med Progress Note - 3\nDescription:  Sample progress note - Gen Med.\n(Medical Transcription Sample Report)\nCHIEF COMPLAINT: Followup on diabetes mellitus, hypercholesterolemia, and sinusitis.\nSUBJECTIVE:  A 70-year-old female who was diagnosed with diabetes mellitus last fall.  She has been checking her Accu-Cheks generally once a day, and they range from 82 to a high of 132.  She feels well.  She walks 1-2 miles most days.  If the weather is bad, they go to a local mall.  Otherwise they walk outside.  She does complain of some sinus congestion and drainage for the last several days.  She is up-to-date on her mammogram that she had at the Baylis Building.  She sees Dr. Cheng for her gynecological care.  She is a nonsmoker.  Denies abdominal pain, nausea, vomiting, diarrhea, constipation, blood in her urine, blood in her stools.  She has nocturia x 1.  Denies swelling in her ankles.  She checks he

## Pipe 2: Segmentation to extract the ROS section

Function definition

In [57]:
"""Functions Required for ROS Segmentation"""
#====================================================================
import pandas as pd
def sectag_to_regex(header_file_path, seg_col, header_col):
  header_df = pd.read_csv(header_file_path)
  header_df = header_df.drop_duplicates()
  headers = header_df[header_col].tolist()
  header_patterns = [f'^{header}[\n:]' for header in headers]
  return header_patterns, header_df[seg_col].tolist()

#====================================================================
import re
def find_segs(note, header_patterns, seg_names):
  segs = {}

  # Find the section headers and their start positions
  for i, pattern in enumerate(header_patterns):
    for m in re.finditer(pattern, note.lower(), re.MULTILINE):
      seg_head = (note[m.span()[0]:m.span()[1]], m.span()[0])
      if seg_head not in segs:
        segs[seg_head] = []  # A seg head can have multiple general seg names

      segs[seg_head].append(seg_names[i]) 

  segs = [[k[0], segs[k], k[1]] for k in segs.keys()]
  segs = sorted(segs, key=lambda x: x[2])
  
  # Find the entir sections and their start and end positions
  for i in range(len(segs)):
    if i == len(segs)-1:
      segs[i].append(len(note))
    else:
      segs[i].append(segs[i+1][2])

  return segs

#====================================================================
def ros_seg(note, segs):
  ros_data = []   # ros text + start position
  right_after_ros = False # flag sections after ROS
  for seg in segs:
      section_names = seg[1]
      section_content = note[seg[2]:seg[3]]  

      #---------------------------------------------------------------
      # Handle hiearchical subsections within ROS if exist
      if right_after_ros:
        if any("review" in item for item in section_names):
          # If there is a review section, append it to the ros_data
          ros_data[0] += section_content
        else:
          right_after_ros = False  # Consider it goes to another section
     
      #---------------------------------------------------------------
      if 'review_of_systems' in section_names:
          ros_data = [section_content, seg[2]]
          right_after_ros = True

  return ros_data

Extraction

In [58]:
header_patterns, seg_names = sectag_to_regex(r'SecTag.csv', 'kmname', 'str')
for filename, note in dflow.items():
    segs = find_segs(note, header_patterns, seg_names)
    ros_data = ros_seg(note, segs)        
    dflow[filename] = ros_data

#print(segs)
dflow


{'sample_71.txt': [],
 'sample_1568.txt': [],
 'sample_403.txt': [],
 'sample_2790.txt': ['REVIEW OF SYSTEMS:  No chills, fever, shakes or tremors.  Denies chest pain palpitations, hemoptysis, shortness of breath, nausea, vomiting, diarrhea, constipation or hematemesis.  The patient reports that his last bowel movement was on 08/30/2007.  No urological symptoms such as dysuria, frequency, incomplete bladder emptying or voiding difficulties.  The patient does report that he has occasional intermittent &quot;numbness and tingling&quot; of his hands bilaterally as he has a history of bilateral carpal tunnel syndrome.  He denies any history of seizure disorders, but he did report that he had some momentary dizziness earlier, but that has since resolved.\n',
  2446],
 'sample_226.txt': ['REVIEW OF SYSTEMS: The patient prior to today has been very well without any signs or symptoms of viral illness, but yesterday he began to experience symptoms of nausea, had an episode of vomiting last nigh

## Pipe 3: LLM to extract diseases, symptoms and body systems and their positive/negative status

Function to remove unnescessary characters in json output from LLMs

In [59]:
import re
def remove_char_json(text):
    pattern = r'\[.*\]'
    match = re.search(pattern, text, re.DOTALL)
    result = match.group()
    return result

Extract diseases and convert to json

In [60]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("user", """{ros_text}"""),
    ("user", "convert to json, remove any unnecessary text and make sure the output starts with [")
])

llm1=ChatOllama(
    model="ros_extract"
)

chain = prompt|llm1

In [61]:
import json
for filename in dflow.keys():
    if dflow[filename] != []:
        ros_text = dflow[filename][0]
        ros_extract = chain.invoke(ros_text)
        
        try:
            ros_extract_json = json.loads(ros_extract.content)
        except:
            try:
                # Try again, assuming the error was due to the AI output being not a JSON
                ros_extract2 = llm1.invoke(f"Convert to json, remove any unnecessary text, for example ```, and make sure the output starts with [: {ros_extract.content}")
                ros_extract2_corrected = remove_char_json(ros_extract2.content)
                ros_extract_json = json.loads(ros_extract2_corrected)
            except:
                # If it still fails, just give an empty JSON list
                ros_extract_json = []
        dflow[filename].append(ros_extract_json)

dflow

{'sample_71.txt': [],
 'sample_1568.txt': [],
 'sample_403.txt': [],
 'sample_2790.txt': ['REVIEW OF SYSTEMS:  No chills, fever, shakes or tremors.  Denies chest pain palpitations, hemoptysis, shortness of breath, nausea, vomiting, diarrhea, constipation or hematemesis.  The patient reports that his last bowel movement was on 08/30/2007.  No urological symptoms such as dysuria, frequency, incomplete bladder emptying or voiding difficulties.  The patient does report that he has occasional intermittent &quot;numbness and tingling&quot; of his hands bilaterally as he has a history of bilateral carpal tunnel syndrome.  He denies any history of seizure disorders, but he did report that he had some momentary dizziness earlier, but that has since resolved.\n',
  2446,
  [{'extract': 'chills', 'status': 'negative'},
   {'extract': 'fever', 'status': 'negative'},
   {'extract': 'shakes', 'status': 'negative'},
   {'extract': 'tremors', 'status': 'negative'},
   {'extract': 'chest pain', 'status

## Pipe 4: LLM to identify the systems

Function to clean up and organize ros entities captured from LLMs output: remove unnecessary characters and group them into ros category

In [62]:
import re
def regexp_ros(text):
  pattern = r"-->\s*(?P<ros>[^(\n]+)"
  m = re.search(pattern, text, re.I)
  if m:
    return m.group('ros').strip()
  else:
    return 'None'

Identify the systems

In [63]:
llm2=ChatOllama(
    model="ros_classify"
)

In [64]:
for filename in dflow.keys():
    if dflow[filename] != []:
        ros_extracts = dflow[filename][2]
        ai_output = "" # Track the AI output before regex process
        for i in ros_extracts:
            ros_cat = llm2.invoke(i['extract']).content
            ai_output += f'###{ros_cat}\n'
            i['sys']=regexp_ros(ros_cat)
        
        dflow[filename].append(ai_output)
dflow

{'sample_71.txt': [],
 'sample_1568.txt': [],
 'sample_403.txt': [],
 'sample_2790.txt': ['REVIEW OF SYSTEMS:  No chills, fever, shakes or tremors.  Denies chest pain palpitations, hemoptysis, shortness of breath, nausea, vomiting, diarrhea, constipation or hematemesis.  The patient reports that his last bowel movement was on 08/30/2007.  No urological symptoms such as dysuria, frequency, incomplete bladder emptying or voiding difficulties.  The patient does report that he has occasional intermittent &quot;numbness and tingling&quot; of his hands bilaterally as he has a history of bilateral carpal tunnel syndrome.  He denies any history of seizure disorders, but he did report that he had some momentary dizziness earlier, but that has since resolved.\n',
  2446,
  [{'extract': 'chills',
    'status': 'negative',
    'sys': 'Constitutional Symptoms'},
   {'extract': 'fever',
    'status': 'negative',
    'sys': 'Constitutional Symptoms'},
   {'extract': 'shakes', 'status': 'negative', 's

## Pipe 5: Output to files

In [65]:
with open(f'{output_folder_path}/dflow.txt', 'w') as f:
    f.write(str(dflow))

with open(f'{output_folder_path}/results.csv', 'w') as f:
    f.write("Filename,Extract,Status,Sys\n")
    for filename in dflow.keys():
        if dflow[filename] != []:
            ros_results = dflow[filename][2]
            if len(ros_results) == 0:
                f.write(f"{filename},NA,NA,NA\n")
            else:
                for i in ros_results:
                    f.write(f"{filename},{i['extract']},{i['status']},{i['sys']}\n")